# Detection of tree stems with Pyoints
In the following, we try to detect stems in a forest using a three dimnesional point cloud generated by a terrestrial laser scanner.
The basic idea is to:
1. Loading of point cloud data
2. Calculation of the height above ground
3. Fitering of stem points
4. Fitting of stem vectors

In [ ]:
import numpy as np

from pyoints import (
	storage,
	filters,
	interpolate,
)

In [ ]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt 

%matplotlib inline

## Fitering of stem points

Load the a LAS file of a forest.

In [ ]:
lasReader = storage.LasReader('forest.las')

Get some information of the point cloud.

In [ ]:
print('number of points:')
print(len(lasReader))
print('projection:')
print(lasReader.proj.proj4)
print('transformation matrix:')
print(lasReader.t)
print('origin:')
print(lasReader.t.origin)
print('extent:')
print(lasReader.extent)

If everything is fine we load the points.

In [ ]:
las = lasReader.load()

We recieve a numpy record array of the point cloud. So we inspect its properties first.

In [ ]:
print('shape:')
print(las.shape)
print('attributes:')
print(las.dtype)
print('projection:')
print(las.proj.proj4)
print('transformation:')
print(las.t)
print('data')
print(las)

## Calculation of the height above ground
Since we LAS file does just provide altitude values instead of heights above ground, we select points representing the ground first, to fit a digital elevation model (DEM). Using the DEM we calculate the heiht of each point above ground.

We use a DEM filter of a resolution of 0.5 m. The filter selects low points and garanties a horizontal point distance of at least 0.5 m and a maximal altitude change between neighbored points of 50 degree.

In [ ]:
grd_ids = filters.dem_filter(las.coords, 0.5, max_angle=50)
print(grd_ids)

We have recieved a list of point indices, which can be used to select the desired representative ground points. So we plot them first.

In [ ]:
grd_coords = las[grd_ids].coords
plt.scatter(grd_coords[:, 0], grd_coords[:, 1], color='black')
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.show()


We can see that the points are distributed almost uniformly.

Now we fit a DEM using a nearest neigbour interpolator.

In [ ]:
xy = las.coords[grd_ids, :2]
z = las.coords[grd_ids, 2]
dem = interpolate.KnnInterpolator(xy, z)

Finally we calculate the height above ground. And add a height attribute to the LAS file

In [ ]:
height = las.coords[:, 2] - dem(las.coords)
las = las.add_fields([('height', float)], data=[height])
print(las.dtype)

## Fitering of stem points
We will filter the points subsequently until only points associated with stems remain.

We will focus on points with height above ground greater 0.5 m only.

In [ ]:
f_ids = np.where(las.height > 0.5)[0]
las = las[f_ids]
print(len(las))

We filter the point cloud using a small filter radius. Only a subset of points with a point distance of at least 10 cm is kept.

In [ ]:
f_ids = list(filters.ball(las.indexKD(), 0.1))
las = las[f_ids]
print(len(las))

Let's take a look at the remaining point cloud.

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = plt.axes(projection='3d', aspect='equal')
ax.set_zlim(lasReader.extent.min_corner[2], lasReader.extent.max_corner[2])

ax.scatter(*grd_coords.T, color='black')
ax.scatter(*las.coords.T, c=las.height, cmap='coolwarm', marker='.')
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
plt.show()

We can see four trees with linear stems. But there are a lot of points associated with branches or leaves. Let's copy that point cloud for later viusalisation

In [ ]:
las_trees = las.copy()

To filter the stems we only keep points with a lot of neighbors to reduce noise.

In [ ]:
count = las.indexKD().ball_count(0.3)
las = las[count > 10]
print(len(las))

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = plt.axes(projection='3d', aspect='equal')
ax.set_zlim(lasReader.extent.min_corner[2], lasReader.extent.max_corner[2])

ax.scatter(*grd_coords.T, color='black')
ax.scatter(*las_trees.coords.T, c=las_trees.height, cmap='coolwarm', marker='.')
ax.scatter(*las.coords.T, color='black')
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
plt.show()

That's mutch better, but be are interrested in position of the stems only. So, we will filter with a radius of 1 m to remove similar points. This results in a point cloud with point distances of at least 1 m.

In [ ]:
f_ids = list(filters.ball(las.indexKD(), 1.0))
las = las[f_ids]
print(len(las))

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = plt.axes(projection='3d', aspect='equal')
ax.set_zlim(lasReader.extent.min_corner[2], lasReader.extent.max_corner[2])

ax.scatter(*grd_coords.T, color='black')
ax.scatter(*las_trees.coords.T, c=las_trees.height, cmap='coolwarm', marker='.')
ax.scatter(*las.coords.T, color='black')
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
plt.show()

For dense point clouds, the filtering technique results in point distances between 1 m and 2 m. Thus, we can assume that linear arranged points should have 2 to 3 neighboring points within a radius of 1.5 m.

In [ ]:
count = las.indexKD().ball_count(1.5)
mask = np.all((count >= 2, count <= 3), axis=0)
las = las[mask]
print(len(las))

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax = plt.axes(projection='3d', aspect='equal')
ax.set_zlim(lasReader.extent.min_corner[2], lasReader.extent.max_corner[2])

ax.scatter(*grd_coords.T, color='black')
ax.scatter(*las_trees.coords.T, c=las_trees.height, cmap='coolwarm', marker='.')
ax.scatter(*las.coords.T, color='black')
ax.set_xlabel('X (m)')
ax.set_ylabel('Y (m)')
ax.set_zlabel('Z (m)')
plt.show()